In [72]:
log = print

In [73]:
arr = list(map(int, """
17 39 135 137 33 165 9 35 139 69 37 153 99 195 77 105 79 193 95 125 61 91 143 75 198 107 121 55 113 7 3 101 45 13 59 167 109 169 23 57 65 89 161 29 197 177 63 175 181 133 123 83 31 103 11 71 149 187 51 145 141 19 15 179 93 87 171 5 21 131 157 49 147 43 151 97 47 159 155 115 53 117 119 129 73 173 81 189 191 127 111 85 67 163 183 41 185 25 27
""".strip().split()))

In [74]:
# arr

In [75]:
def distribute_to_equal_subset_sum(arr):
    total_sum = sum(arr)

    if total_sum % 2 != 0:
        return None

    target_sum = total_sum // 2
    n = len(arr)

    # Initialize the dynamic programming table
    dp = [False] * (target_sum + 1)
    dp[0] = True

    for i in range(n):
        for j in range(target_sum, arr[i] - 1, -1):
            if not dp[j]:
                dp[j] = dp[j - arr[i]]

    if not dp[target_sum]:
        return None

    # Backtrack to find the subset
    subset = []
    j = target_sum
    for i in range(n - 1, -1, -1):
        if j >= arr[i] and dp[j - arr[i]]:
            subset.append(arr[i])
            j -= arr[i]

    return subset

    c = Counter(left)
    right = []
    for x in arr:
        log(x, c[x])
        if c[x] > 0:
            c[x] -= 1
            continue
        right.append(x)

    log(sum(left), sum(right), sum(arr))
    assert sum(left) == sum(right)
    assert sum(left) + sum(right) == sum(arr)
    assert sorted(left + right) == sorted(arr)
    return left, right

In [78]:
# distribute_to_equal_subset_sum(arr)